In [ ]:
!pip install lightgbm
!pip install imblearn
!pip install wandb

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
# import torch
# from torch import nn
import lightgbm as lgb
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import wandb
import pickle
from utils import *

# Load and preprocess data(Feature Engineering using RF.ipynb)

In [ ]:
data_df=load_processed_csv(train=True)
data_df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature63,feature64,feature65,feature66,feature67,feature68,feature69,feature70,feature71,label
0,0.302245,0.0,0.2,0.0,1.0,1.0,0.0,0.0,0.0,0.125,...,0,0,2,1,0,0,1,0,0,3
1,-0.224298,0.0,0.3,0.0,1.0,1.0,1.0,0.0,0.0,0.125,...,0,0,1,1,4,5,0,0,0,3
2,0.206510,0.0,0.7,0.0,1.0,1.0,0.0,0.0,0.0,0.000,...,0,0,1,1,4,0,0,0,0,3
3,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3
4,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,0.250,...,0,0,1,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9553,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9554,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1
9555,-0.750840,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.250,...,5,1,2,2,0,1,0,0,3,1


In [ ]:
# plt.figure(figsize=(50,50))
# sb.heatmap(data_df.corr(), xticklabels=data_df.corr().columns, yticklabels=data_df.corr().columns, cmap='RdYlGn', center=0, annot=True)


# Define KFolder and light_bgm Function

In [ ]:
def light_bgm_train(X_train,y_train,X_test,y_test,params=None):
    train_data=lgb.Dataset(X_train,label=y_train)
    validation_data=lgb.Dataset(X_test,label=y_test)
    if params is None:
        params={
            'learning_rate':0.1,
            'lambda_l1':0.1,
            'lambda_l2':0.2,
            'max_depth':10,#default 6
            'objective':'multiclass',
            'num_class':4,  
            # 'max_bin':255,# 大会有更准的效果,更慢的速度
            # 'max_depth': -1,  # 小数据集下限制最大深度可防止过拟合,小于0表示无限制
            # 'num_leaves': 64,  # 大会更准,但可能过拟合
            # 'feature_fraction': 0.8,  # 防止过拟合
            # 'bagging_freq': 5,  # 防止过拟合
            # 'bagging_fraction': 0.8,  # 防止过拟合
            # 'min_data_in_leaf': 21,  # 防止过拟合
            # 'min_sum_hessian_in_leaf': 3.0,  # 防止过拟合
        }
    clf=lgb.train(params,train_data,valid_sets=[validation_data])
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = light_bgm_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

# Confusion Matrix and Report

In [ ]:
X = data_df.iloc[:, :-1]
Y = data_df["label"]
print("original distribution:",Counter(Y))
#upsamlping 
smo = SMOTE(random_state=42)
X_smo, Y_smo = smo.fit_resample(X, Y)
print("smote distribution:",Counter(Y_smo))
X,Y=X_smo, Y_smo#using smote dataset

original distribution: Counter({3: 5996, 1: 1597, 2: 1209, 0: 755})
smote distribution: Counter({3: 5996, 1: 5996, 2: 5996, 0: 5996})


In [ ]:
!wandb login 5bc70e88e5aac2b0cd097233114e75b47f6888e6 --relogin 

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# wandb.init(project="light_gbm", entity="cz4041",id='num_leaves_2_200_depth=-1')
params={ 'learning_rate':0.1,
            'lambda_l1':0.1,
            'lambda_l2':0.2,
            'max_depth':-1,#default 6
            'num_leaves':128 ,  # 大会更准,但可能过拟合
            'objective':'multiclass',
            'num_class':4, }
wandb.config.update(params)
macro_F1_metric=[]
for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

    # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
    # y_test_pred = model.predict(X_test).argmax(axis=1)
    y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
    y_test=np.array(Y)[test_index]

    score = accuracy_score(y_test, y_test_pred)
    print("Accuracy score is " + str(score))
    report=classification_report(y_test, y_test_pred)
    macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
    with open('LightGBM_data/LightGBM_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
        pickle.dump(model, f)#store as pickle
    print('model saved to LightGBM_data/LightGBM_model_{}.pkl'.format(macro_F1_metric[-1]))
    print(report)

    # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
# wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12586
[LightGBM] [Info] Number of data points in the train set: 19187, number of used features: 72
[LightGBM] [Info] Start training from score -1.384992
[LightGBM] [Info] Start training from score -1.381044
[LightGBM] [Info] Start training from score -1.387911
[LightGBM] [Info] Start training from score -1.391258
[1]	valid_0's multi_logloss: 1.23158
[2]	valid_0's multi_logloss: 1.11204
[3]	valid_0's multi_logloss: 1.01327
[4]	valid_0's multi_logloss: 0.930998
[5]	valid_0's multi_logloss: 0.860237
[6]	valid_0's multi_logloss: 0.797342
[7]	valid_0's multi_logloss: 0.740033
[8]	valid_0's multi_logloss: 0.690954
[9]	valid_0's multi_logloss: 0.646399
[10]	valid_0's multi_logloss: 0.606179
[11]	valid_0's multi_logloss: 0.571775
[12]	valid

In [ ]:
with open(('LightGBM_data/LightGBM_model_0.98.pkl'), 'rb+') as f:
    model = pickle.load(f)#load model
generate_sumbit_csv(model,'lightbgm_upsampling')

save to data/test_submit_lightbgm_upsampling_02-02-2022_17_15_11_508765.csv
distribution:
1 1196
2 3527
3 1385
4 17748


True

In [ ]:
# #########################################################################################
# # 模型 lightgbm
# #########################################################################################

# # 准备数据
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
# # X_valid, X_test, y_valid, y_test = train_test_split(X_test,y_test,test_size=0.5)
# # 训练
# btime = datetime.datetime.now() 
# clf=light_bgm_train(X_train,y_train,X_test,y_test)
# print ('all tasks done. total time used:%s s.\n\n'%((datetime.datetime.now() - btime).total_seconds()))

# # 1、AUC
# y_pred_pa = clf.predict(X_test)  # !!!注意lgm预测的是分数，类似 sklearn的predict_proba
# # y_test_oh = label_binarize(y_test, classes= [0,1,2,3])
# # print ('调用函数auc：', roc_auc_score(y_test_oh, y_pred_pa, average='micro'))

# #  2、混淆矩阵
# y_pred = y_pred_pa.argmax(axis=1)
# confusion_matrix(y_test, y_pred)

# #  3、经典-精确率、召回率、F1分数
# precision_score(y_test, y_pred,average='micro')
# recall_score(y_test, y_pred,average='micro')
# f1_score(y_test, y_pred,average='micro')

# # 4、模型报告
# print(classification_report(y_test, y_pred))

# # 保存模型
# with open('LightGBM_data/LightGBM_model.pkl', 'wb') as f:
#     pickle.dump(clf, f)
#     print('model saved')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f92fbbcf-fa92-4caa-ad57-5cd0b8c2766a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>